In [14]:
import requests
from bs4 import BeautifulSoup, Tag
from concurrent.futures import ThreadPoolExecutor, as_completed
import random
import time

In [4]:
test_urls = [
    "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes/627367-peoplesoft-hcm-hrms-functional-consultant-resume-3",
    "https://www.hireitpeople.com/resume-database/80-peoplesoft-resumes/613216-peoplesoft-functional-consultant-business-analyst-resume-atlanta-ga-4",
    "https://www.hireitpeople.com/resume-database/69-help-desk-support-resumes/614271-help-desk-support-trading-floor-support-resume-new-york-ny",
    "https://www.hireitpeople.com/resume-database/69-help-desk-support-resumes/606776-computer-technician-field-associate-resume-new-york-ny-11",
]

In [19]:
def scrape_resume(url: str, session=None) -> dict:
    """Scrape details from an individual resume page, keeping DOM tags without duplication."""
    if session is None:
        session = requests.Session()

    response = session.get(url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, "lxml")
    sections = {}

    target_divs = soup.find_all("div", class_=["media-body", "single-post-body"])

    for div in target_divs:
        # Find all section headers (<u> or <strong> inside <u>)
        u_tags = div.find_all("u")

        for i, u_tag in enumerate(u_tags):
            key = u_tag.get_text(strip=True).replace(":", "").upper()

            content_tags = []
            # Walk siblings of the parent <p> (or <u> parent) until the next header
            for sibling in u_tag.parent.next_siblings:
                if isinstance(sibling, Tag):
                    # Stop if we reach another <u> tag
                    if sibling.find("u"):
                        break
                    # Append only top-level sibling tags (no recursion)
                    content_tags.append(sibling)

            sections[key] = content_tags

    return sections


res_dict = scrape_resume(test_urls[3])

In [21]:
res_dict.keys()

dict_keys(['OBJECTIVE', 'TECHNICAL SKILLS', 'PROFESSIONAL EXPERIENCE'])

In [23]:
res_dict["OBJECTIVE"]

[<ul>
 <li>To obtain a challenging position in the Information Technology Field that will Allow me to utilize my skills with a growing company.</li>
 </ul>]

In [24]:
res_dict["PROFESSIONAL EXPERIENCE"]

[<p><strong>Confidential, New York, NY</strong></p>,
 <p><strong>Consultant/NOC</strong></p>,
 <p><strong>Responsibilities:</strong></p>,
 <ul>
 <li>Working in the network operation center on the bridge for the field tech to call in for support and document all data</li>
 <li>Creating tickets for many technical issue and escalating the ticket to the correct department</li>
 <li>Investigating when having technical issue and report to upper management</li>
 <li>Computer tech on wheels troubleshoot and repair hardware, software and network, break fix laptop and desktop remove virus and backup files</li>
 <li>Configuring of TCP/IP and IP addressing, installing and upgrading multiply kinds of software</li>
 </ul>,
 <p><strong>Confidential, New York, NY</strong></p>,
 <p><strong>Computer Technician Field Associate</strong></p>,
 <p><strong>Responsibilities:</strong></p>,
 <ul>
 <li>Break fix, attending to customers location servicing all Dell desktop, laptop and Gateway computers replacing a